In [ ]:
# gemini generated
import nidaqmx
from nidaqmx.constants import AcquisitionType
import numpy as np
import time

# --- 設定量測參數 ---
# 您的 NI DAQ 設備與類比輸入通道名稱。
# 請根據您的實際設備和通道名稱進行修改。例如："Dev1/ai0" 或 "cDAQ1Mod1/ai0"
NI_DAQ_CHANNEL = "Dev1/ai0" 
SAMPLE_RATE = 1000             # 採樣率 (Hz)
MEASUREMENT_DURATION = 5.0     # 量測持續時間 (秒)

# 計算需要採集的總樣本數
number_of_samples = int(SAMPLE_RATE * MEASUREMENT_DURATION)

print(f"準備從通道 '{NI_DAQ_CHANNEL}' 採集數據...")
print(f"採樣率: {SAMPLE_RATE} Hz")
print(f"量測時間: {MEASUREMENT_DURATION} 秒")
print(f"總共採集樣本數: {number_of_samples} 個")

準備從通道 'Dev1/ai0' 採集數據...
採樣率: 1000 Hz
量測時間: 5.0 秒
總共採集樣本數: 5000 個


In [ ]:
# --- 創建並執行 NI DAQ Task ---
try:
    # 'with' 語句確保 Task 在程式結束或發生錯誤時會被自動關閉
    with nidaqmx.Task() as task:
        # 添加一個類比輸入電壓通道
        task.ai_channels.add_ai_voltage_chan(NI_DAQ_CHANNEL)
        
        # 設定採樣時脈與採集模式。
        # AcquisitionType.FINITE 模式會讓程式採集指定數量的樣本後停止。
        task.timing.cfg_samp_clk_timing(
            rate=SAMPLE_RATE, 
            sample_mode=AcquisitionType.FINITE,
            samps_per_chan=number_of_samples
        )
        
        # 啟動 Task
        task.start()
        
        # 等待數據採集完成
        print("\n開始量測... 請稍候...")
        task.wait_until_done(timeout=MEASUREMENT_DURATION + 2.0) # 給予額外的時間裕度
        
        # 讀取所有採集到的數據
        acquired_data = task.read(number_of_samples_per_channel=number_of_samples)
        
        # 將數據轉換為 numpy 陣列，方便後續處理
        data_array = np.array(acquired_data)
        
        print("\n量測完成！\n")
        
        # --- 數據分析與結果列印 ---
        if data_array.size > 0:
            print("--- 量測數據概要 ---")
            print(f"總共獲得 {data_array.size} 個數據點。")
            print(f"數據平均值: {np.mean(data_array):.4f} V")
            print(f"數據最小值: {np.min(data_array):.4f} V")
            print(f"數據最大值: {np.max(data_array):.4f} V")
            
            # 選擇性地列印前10個數據點
            print("\n前10個數據點:")
            print(data_array[:10])
        else:
            print("沒有採集到任何數據。請檢查您的連線和設定。")
            
except nidaqmx.DaqError as e:
    print(f"\nNI DAQ 錯誤: {e}")
    print("請檢查您的 NI DAQ 設備連線、通道名稱以及配置。")
except Exception as e:
    print(f"\n發生了其他錯誤: {e}")